In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import yfinance as yf
from machine_learning_finance import analyze_trades, calc_probabilties_without_lookahead, plot_backtest_analysis, metrics_to_dataframe
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)
windows = [300, 600, 900, 1500]

symbol = "XLE"
file = f"../backtests/backtest_inverse_{symbol}_365_h0.9_l0.1.csv"
period = 365
pd.options.display.max_rows = None


def plot_win_loss(file, metrics):
    ledger = pd.read_csv(file)
    # Set the style for the plots
    sns.set(style='whitegrid')

    # Create a bar plot for profit and loss stats
    profit_stats, loss_stats = metrics['profit_stats'], metrics['loss_stats']
    profit_loss_df = pd.DataFrame([profit_stats, loss_stats], columns=['min', 'max', 'mean', 'median', 'std'], index=['profit', 'loss'])

    plt.figure(figsize=(12, 6))
    ax = sns.barplot(data=profit_loss_df.transpose(), palette='muted')
    ax.set_title('Profit and Loss Statistics')
    ax.set_ylabel('Value')

    # Show the plot
    plt.show()

def analyze_and_graph(file, symbol, period, df):
    ledger = pd.read_csv(file)

    hist_df = df.head(len(df)-period)
    hist_df = hist_df.tail(period*4)
    df = df.tail(period)
    df = calc_probabilties_without_lookahead(df, hist_df)
    plot_backtest_analysis(df, ledger, inverse=symbol)

    return ledger, df

ticker_obj = yf.download(tickers=symbol)
df = pd.DataFrame(ticker_obj)
#plot_win_loss(file, metrics)

#print(ledger)

#print(metrics_df)
#df.sort_index(ascending=False)

[*********************100%***********************]  1 of 1 completed


In [2]:
ledger, df = analyze_and_graph(file, symbol, 365*4, df)    

# Display metrics as text
metrics = analyze_trades(ledger, symbol, period)
metrics["file"] = file
metrics_df = metrics_to_dataframe(metrics) 

merged_df = ledger.merge(df, left_on='Date', right_index=True, how='left')
merged_df[["Date", "Price", "Product", "Side", "Action","prob_above_trend"]]

/Users/jmordetsky/machine_learning_finance/src/machine_learning_finance/timeseries_analytics.py:24: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/jmordetsky/machine_learning_finance/src/machine_learning_finance/timeseries_analytics.py:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/jmordetsky/machine_learning_finance/src/machine_learning_finance/timeseries_analytics.py:26: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

[*********************100%***********************]  1 of 1 completed


,Date,Price,Product,Side,Action,prob_above_trend
0,2020-03-27,28.613300,XLE,long,enter,1.000000
1,2020-06-03,42.147300,XLE,long,exit,0.000650
2,2020-06-03,193.263506,DUG,long,enter,0.000650
3,2020-07-09,245.833994,DUG,long,exit,0.993930
4,2020-07-09,34.784399,XLE,long,enter,0.993930
5,2020-08-05,37.804300,XLE,long,exit,0.097816
6,2020-08-05,202.353506,DUG,long,enter,0.097816
7,2020-09-23,288.456006,DUG,long,exit,0.992640
8,2020-09-23,30.512099,XLE,long,enter,0.992640
9,2020-11-24,40.713101,XLE,long,exit,0.000641


In [4]:
df.tail(10)


,Open,High,Low,Close,Adj Close,Volume,prob_above_trend,weighted-volume,trend,long_entry,long_exit,short_entry,short_exit
Date,,,,,,,,,,,,,
2023-05-04,79.019997,79.820000,77.870003,78.110001,78.110001,26430500,0.977544,2.064486e+09,85.745772,NaN,NaN,NaN,NaN
2023-05-05,80.269997,80.860001,79.790001,80.230003,80.230003,19533600,0.927706,1.567181e+09,85.737760,NaN,NaN,NaN,NaN
2023-05-08,81.400002,81.750000,80.220001,80.230003,80.230003,16355900,0.932191,1.312234e+09,85.729748,NaN,NaN,NaN,NaN
2023-05-09,79.599998,81.019997,79.430000,80.260002,80.260002,20710600,0.927931,1.662233e+09,85.721736,NaN,NaN,NaN,NaN
2023-05-10,80.660004,80.820000,78.750000,79.349998,79.349998,18026000,0.960861,1.430363e+09,85.713724,NaN,NaN,NaN,NaN
2023-05-11,78.230003,78.809998,77.800003,78.400002,78.400002,19084100,0.980460,1.496193e+09,85.705713,NaN,NaN,NaN,NaN
2023-05-12,78.790001,79.320000,77.949997,78.519997,78.519997,18952700,0.980848,1.488166e+09,85.697701,NaN,NaN,NaN,NaN
2023-05-15,78.809998,79.370003,78.190002,78.919998,78.919998,16272300,0.977889,1.284210e+09,85.689689,NaN,NaN,NaN,NaN
2023-05-16,78.550003,78.889999,76.860001,76.980003,76.980003,22255600,0.994314,1.713236e+09,85.681677,NaN,NaN,NaN,NaN
